In [12]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6,7,8"
print(torch.cuda.device_count())

3


In [11]:
import json
from datasets import load_dataset
import itertools

In [3]:
with open("train_focus.json", "r") as read_file:
    train_data = json.load(read_file)
with open("valid_focus.json", "r") as read_file:
    valid_data = json.load(read_file)
    
train_dataset = []
for obj in train_data['data']:
    train_dataset.append(obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))])   
    
valid_dataset = []
for obj in valid_data['data']:
    valid_dataset.append(obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))])
    
test_dataset = train_dataset[-len(valid_dataset) : ]
train_dataset = train_dataset[ : -len(valid_dataset)]

train_df = list(itertools.chain.from_iterable(train_dataset))
test_df = list(itertools.chain.from_iterable(test_dataset))
valid_df = list(itertools.chain.from_iterable(valid_dataset))

In [6]:
average = sum(map(lambda x: len(x), train_dataset)) / len(train_dataset)
print(f"Average: {average}")
print(f"top_k = {int(average)}")

Average: 11.267542454341557
top_k = 11


In [14]:
df = train_df + test_df + valid_df

In [15]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
def sbert_encoder(df):
    model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
    model = model.to('cuda')
    
    def get_embedding(sentence):
        return model.encode(sentence)

    embeddings = list(map(get_embedding, df))
    embeddings = np.array(embeddings)
    np.savetxt(f"distil_roberta_embeddings.txt", embeddings)
    return embeddings

In [16]:
embs = sbert_encoder(df)

In [17]:
embs = np.loadtxt(f"distil_roberta_embeddings.txt")

In [18]:
embs.shape

(151942, 768)

In [19]:
embs

array([[-0.02883341, -0.01815435,  0.01111846, ...,  0.01524358,
         0.01135345,  0.03803908],
       [-0.00880675, -0.0328324 , -0.00595096, ...,  0.04789126,
         0.07005066,  0.01678892],
       [-0.03496892,  0.01408572, -0.0159178 , ...,  0.04205874,
         0.00938218, -0.00708636],
       ...,
       [-0.02731626, -0.0180053 ,  0.02140533, ...,  0.02867802,
         0.05854803, -0.01566559],
       [-0.00076037, -0.02560225,  0.01002467, ...,  0.03856468,
         0.05161415,  0.03129318],
       [-0.0329641 , -0.03726761, -0.00084354, ...,  0.04821602,
        -0.00181708,  0.00724081]])

In [3]:

import json
from datasets import load_dataset
import itertools

with open("/cephfs/home/ledneva/focus/utils/train_focus.json", "r") as read_file:
    train_data = json.load(read_file)
with open("/cephfs/home/ledneva/focus/utils/valid_focus.json", "r") as read_file:
    valid_data = json.load(read_file)

train_dataset = []
for obj in train_data['data']:
    len_dialog = len(obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))])
    train_dataset.append({"utterance" : obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))],
                               "speaker" :  [i % 2 for i in range(len_dialog)]})                                                     

valid_dataset = []
valid_speaker = []
for obj in valid_data['data']:
    len_dialog = len(obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))])
    valid_dataset.append({"utterance" : obj['utterance'][-1]['dialogue' + str(len(obj['utterance']))],
                   "speaker" :  [i % 2 for i in range(len_dialog)]})        

test_dataset = train_dataset[-len(valid_dataset) : ]
train_dataset = train_dataset[ : -len(valid_dataset)]

In [4]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(11484, 1000, 1000)